In [32]:
import pandas as pd
import numpy as np

from itertools import chain

from pathlib import Path
from functools import reduce

In [2]:
path = Path('./corona_data_sets_modified') # use your path
all_files = list(path.rglob("*.csv"))

In [4]:
d = {}

for p in all_files:
    df = pd.read_csv(p, index_col=None, header=0, converters={'ags2': str, 'ags5': str})
    #df = df.drop(['bundesland', 'kreis', '_id'], 1)
    try:
        df = df.drop(['id'], 1) # only few dbs have the 'id' column
    except:
        pass
    d[p.stem] = df

In [5]:
d.keys()

dict_keys(['reachability_data_modified', 'training_situation', 'hospital_data', 'households', 'traffic', 'publicFinances', 'population', 'commuter_data_modified', 'tourism_data_modified', 'company_information', 'labor_market_development', 'point_of_interest', 'spatialPlanning', 'land_use_data', 'private_finance'])

In [7]:
{k: len(set(v["ags5"])) for k, v in d.items()}

{'reachability_data_modified': 401,
 'training_situation': 401,
 'hospital_data': 401,
 'households': 401,
 'traffic': 401,
 'publicFinances': 401,
 'population': 401,
 'commuter_data_modified': 401,
 'tourism_data_modified': 401,
 'company_information': 401,
 'labor_market_development': 401,
 'point_of_interest': 401,
 'spatialPlanning': 401,
 'land_use_data': 401,
 'private_finance': 401}

In [29]:
dataFrames = [v for v in d.values()]
col = []

total_number_of_columns = 1 # for ags2 (ags 5 becomes index, so not counted as a column)
for df in dataFrames:
    total_number_of_columns += len(df.columns) - 2 # dono't double count ags2 and ags 5 for every table
    col.append(df.columns.values.tolist())
    
print('Total number of columns: ', total_number_of_columns)


Total number of columns:  188


In [9]:
df_final = dataFrames[0].sort_values(by=['ags5'])
for df in dataFrames[1:]:
    for col in df.columns:
        df_final[col] = df.sort_values(by=['ags5'])[col]

# set the index
df_final.set_index('ags5', inplace=True, drop=True)
df_final

,ags2,kreis,supermarkets_population,supermarkets_average_distance,public_transport availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,Unnamed: 0,number_of_students,...,median_income,purchasing_power_per_household,purchasing_power_per_person,debtor_quota,household_in_income_calss_1,household_in_income_calss_2,household_in_income_calss_3,household_in_income_calss_4,household_in_income_calss_5,household_in_income_calss_6
ags5,,,,,,,,,,,,,,,,,,,,,
01001,01,"Flensburg, Stadt",92,500,35,240,2901,240,0,11610,...,2986,34496,19556,16.0,13209,12385,13092,5441,3096,3497
01002,01,"Kiel, Landeshauptstadt",92,460,37,268,2037,265,1,23898,...,3304,35246,19612,12.1,22126,38523,39483,18567,11162,7841
01003,01,"Lübeck, Hansestadt",90,532,37,297,1927,294,2,20478,...,3036,37219,20820,15.1,22881,30159,32571,15805,10999,9039
01004,01,"Neumünster, Stadt",85,588,37,316,1648,313,3,10380,...,2842,38141,19561,17.9,7208,13291,10435,3812,3201,2805
10041,10,Regionalverband Saarbrücken,76,734,52,260,1804,255,318,30091,...,3431,37046,19917,14.4,31031,44117,53073,24122,13716,11119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
09776,09,Lindau (Bodensee),63,1056,36,461,3039,441,313,7001,...,3334,53011,27281,7.6,1899,4382,10928,7859,6828,10105
09777,09,Ostallgäu,59,1418,33,907,3726,837,314,13093,...,3231,52358,25107,6.3,4076,8392,16455,11687,10814,15803
09778,09,Unterallgäu,58,1369,32,855,4129,805,315,12022,...,3297,60441,27461,6.1,4051,6995,15103,12738,10743,15768


In [28]:
col

[Index(['ags2', 'ags5', 'kreis', 'supermarkets_population',
        'supermarkets_average_distance', 'public_transport availability',
        'average_distance_bus_stop', 'average_distance_train_station',
        'average_distance_public_transport'],
       dtype='object'),
 Index(['Unnamed: 0', 'ags2', 'ags5', 'kreis', 'number_of_students'], dtype='object'),
 Index(['Unnamed: 0', 'ags2', 'ags5', 'kreis', 'number_of_hospitals',
        'number_of_hospital_beds', 'number_of_hospital_beds_adj',
        'hospital_patiants'],
       dtype='object'),
 Index(['Unnamed: 0', 'ags2', 'ags5', 'kreis', 'households_of_1_person',
        'households_of_2_person', 'households_of_3_person',
        'households_of_4_person', 'households_of_5_person_or_more',
        'household_with_kids', 'household_with_kids_under_3',
        'household_with_kids_over_3_under_6',
        'household_with_kids_over_6_under_10',
        'household_with_kids_over_10_under_15',
        'household_with_kids_over_15_under_1

In [36]:
df_final.columns

Index(['ags2', 'kreis', 'supermarkets_population',
       'supermarkets_average_distance', 'public_transport availability',
       'average_distance_bus_stop', 'average_distance_train_station',
       'average_distance_public_transport', 'Unnamed: 0', 'number_of_students',
       ...
       'median_income', 'purchasing_power_per_household',
       'purchasing_power_per_person', 'debtor_quota',
       'household_in_income_calss_1', 'household_in_income_calss_2',
       'household_in_income_calss_3', 'household_in_income_calss_4',
       'household_in_income_calss_5', 'household_in_income_calss_6'],
      dtype='object', length=168)

In [37]:
df_final.columns = df_final.columns.str.replace(' ', '_')

In [38]:
df_final.to_csv('./corona_data_sets_modified/df_final.csv')